# 02 Llenado automático de archivos desde una base de datos con python

Alejandro Villegas

2025-05-16

Esta proyecto/demostración está realizado en lenguaje Python. Para su construcción se utilizó el entorno de desarrollo Spyder. Para la cosntrucción de este documento explicativo se utilizó el entorno de desarrollo Jupyter y formato Markdown.

En el repositorio de github de este proyecto encontrarás, los insumos y el código y los productos del código aquí explicados. Esto para facilitar su reproductibilidad.

# Problema a resolver

Puede darse el caso en el cual, tenemos que rellenar un ciertos datos en un gran número de documentos. Y ocurre que los datos con los cuales tenemos que llenar se encuentran en un documento excel. Naturalmente, podriamos llenar uno por uno los documentos, sin embargo, cuando el número de documentos es muy grande, esto nos puede llevar mucho tiempo y esfuerzo. Por fortuna, vivimos en una época en la cual podemos utilizar las posibilidades que ofrece Python para solucionar el problema.

Para este ejercicio, supondré soy el supervisor de varios colegios, y que debo crear examenes personalizados para cada uno de los alumnos bajo mi cargo. Para ello, cada uno de los examenes deberá contener el nombre, los apellidos, el número de plantel, el número de grupo, el turno en el que el alumno asiste a la escuela, la edad y el género del alumno.

En este ejercicio, se planteó una situación donde se requrían examenes, sin embargo, haya otros ámbitos donde esto puede ser útil, como por ejemplo, para formularios de beneficiarios, o similares.

# Insumos.

Para llevar a cabo este ejercicio presupondrémos que contamos con los siguientes insumos:

-   Una base de datos que contenga los datos de cada uno de los alumnos.

-   Una plantilla del examen (o del documento) que se va a llenar en formato word.

Los insumos que se utilizaron en este ejercicio se encuentran en el repositorio de github: LINK

# Metodología.

# Paso 1: Preparación de la base de datos

En este punto se presupone que se tiene una base de datos limpia, que conserva solo las columnas de los datos que nos interesan. Así pues, procedeos a cargar la base, y nos quedaría algo como lo siguiente:

In [2]:
import pandas as pd
base = pd.read_csv(r"C:\Users\javal\OneDrive\Desktop\Portafolio 9\02_llenado_automático_documentos\base.csv")
print(base.head())

               name         lastname  profesor  \
0             Bruno  Resendiz Mateos         1   
1  Cristian Eduardo  Joaquin Petaton         1   
2   Daniela Yoselin   Ortiz Gonzalez         1   
3    Jessica Noelia     Sanchez Lili         1   
4             Erick   Reyes Gonzalez         1   

  Turno al que acudes a la escuela:  ¿Qué edad tienes?    ¿Cuál es tu género?  
0                          Matutino               17.0  Prefiero no responder  
1                          Matutino                NaN              Masculino  
2                          Matutino               19.0               Femenino  
3                          Matutino               17.0               Femenino  
4                          Matutino               17.0              Masculino  


# Paso 2: Preparación de la plantilla.

Esta metodología presupone que el usuario ya tiene lista la plantilla que servirá de base para ser llenada. En este caso el examen. Así pues, lo primero que se debe de hacer es llenar los datos que se van a sustituir en cada documento individual, con un indicador. El indicador se caracteriza por ir entre una doble llave {{}}.

Algo de **vital importancia** es que los nombres de las columnas de la base de datos, debe de coincidir con los nombres de los indicadores, es decir con lo que va entre la doble llave. Por ejemplo, el dato de nombre del alumno, el naame coincide en el indicador y en la columna de la base de datos que muestra los nombres.

Así pues, la plantilla debería quedar como lo siguiente: 
 plantilla.

<div style="text-align: center;">
  <p><strong>Imagen 1: Plantilla</strong></p>
  <img src="imagenes/01_plantilla.png" width="70%">
  <p><em>Fuente: Elaboración propia</em></p>
</div>


Finalmente, establecemos un objeto que contendrá el *string* con la ruta del documento de word que contiene la plantilla.

In [23]:
plantilla = r"C:\Users\javal\OneDrive\Desktop\Portafolio 9\02_llenado_automático_documentos\examen.docx"
print(plantilla)

C:\Users\javal\OneDrive\Desktop\Portafolio 9\02_llenado_automático_documentos\examen.docx


# Paso 3: Lista única de grupos (número de profesores)

Como tenemos distintos grupos, y cada grupo se identifica por el número del profesor, entonces conviene agrupar los exaamenes por grupo. Esto es así porque los examenes se aplicarán a todos los alumnos de un grupo al mismmo tiempo. Entonces para llevar un control e imprimir los examenes de manera adecuada, necesitamos crear dos cosas. La primeraa de ellas es un examen individual con los datos de los alumnos, y la segunda es un archivo que junte todos los examenes individuales de un solo grupo.

El primer paso para lograr esto, es hacer una lista con los valores únicos de los datos que identifican a cada grupo, en este caso, son los datos de los profesore Antes de crear la lista, transformamos el tipo de dato de la columna profesores de numérico a *string*s.


In [7]:
base['profesor'] = base['profesor'].astype(str) #´Primero transformamos el tipo de la columna del número de los profesores
profesores = list(base["profesor"].unique())
print(profesores)

['1', '2', '3']


# Paso 4: Creación de funciones.

Para llevar a cabo nuestro propósito necesitamos crear un conjunto de funciones que ejecuten cada uno de los pasos para llegaar a la meta. Para crear estas funciones primero importamos las librerias que necesitamos.

In [ ]:
from docx import Document #Para manejar los archivos word
import os 
from docx2pdf import convert #Para transformar archvios word a PDF
from pypdf import PdfWriter #Para modificar archivos PDF, como por ejemplo unirlos.

Y comenzamos a crear las funciones

## Función pára llenar el documento word.

In [15]:
def reemplazar_campos (doc, datos_fila):
    for p in doc.paragraphs:
        for key, value in datos_fila.items():
            marcador = f"{{{{{key}}}}}" 
            if marcador in p.text:
                p.text = p.text.replace(marcador, str(value))
    return doc

Notece la línea 5, pues hará que el márcador será igual a la columna con el mismo nombre. Es decir, el marcador de laa plantilla se llenará con el dato de la columna correspondiente.

## Función para generar los examenes individuales.

In [19]:
def generar_exmenes_individuales (plantilla_path, datos, carpeta_destino):
    df = datos
    os.makedirs(carpeta_destino, exist_ok= True)
    for _, fila in df.iterrows():
        doc_temp = Document(plantilla_path)
        doc_temp = reemplazar_campos(doc_temp, fila)
        nombre_archivo = f"{fila['lastname'].replace(' ','_')}.docx"
        ruta_salida = os.path.join(carpeta_destino, nombre_archivo)
        doc_temp.save(ruta_salida)
        print(f"🍀🙊🙊Examenes automaticos guardados con éxito en {ruta_salida}🙊🙊🍀")

## Función para unir los diferentes PDF's en un sólo archivo

In [18]:
def unir_pdfs (carpeta, archivo_salida):
    merger = PdfWriter()
    archivos_pdf = [f for f in os.listdir(carpeta)  if 
f.lower().endswith(".pdf")]
    archivos_pdf.sort()
    for pdf in archivos_pdf:
        ruta_pdf = os.path.join(carpeta,pdf)
        merger.append(ruta_pdf)
        print(f"🎉 Añadido: {pdf}")
        merger.write(archivo_salida)
        merger.close()

# Paso 5: Iteramos sobre cada una de las observaciones de la base de datos.

In [ ]:
for profesor in profesores:
    ruta_salida = fr"C:\Users\javal\OneDrive\Desktop\Portafolio 9\02_llenado_automático_documentos\examenes\{profesor}"
    base2 = base[base["profesor"]== f"{profesor}"]
    generar_exmenes_individuales(
        plantilla_path= plantilla,
        datos = base2,
        carpeta_destino= ruta_salida)
    convert(ruta_salida,ruta_salida)
    ruta_pdf_completo = fr"C:\Users\javal\OneDrive\Desktop\Portafolio 9\02_llenado_automático_documentos\examenes\examenes_completos\{profesor}.pdf"
    unir_pdfs(ruta_salida, ruta_pdf_completo)


Las cosas de vital importancia a tener en cuenta para el paso 5 son:

- Establecer bien las rutas de salida de cada documento de cada alumno.
- Establecer bien la ruta de salidad del pdf completo. Recuerda que cada PDF completo equivale a todos los examene de un sólo profesor.
- En la tercera línea crear una base nueva, dicha base se actualizará con cada iteración, y garantizará que el proceso se haga para cada profesor. Por eso es importante que en el argumento "datos" de la función generar_exmenes (línea 5) corresponda a la nueva base


# Resultados

Para este punto, lo resultados de nuestro código serán los siguientes. Primero, una serie de carpetas, una por cada uno de los profesores. En este caso, sólo tres (profesor 1, 2, y 3). 

<div style="text-align: center;">
  <p><strong>Imagen 2: Carpetas de profesores</strong></p>
  <img src="imagenes/02_carpetas.png" width="70%">
  <p><em>Fuente: Elaboración propia</em></p>
</div>

En segundo lugar, dentro de las carpetas se encontrarán todos los examenes individuales de los alumnos, tanto en formato word, como en formato PDF. 

<div style="text-align: center;">
  <p><strong>Imagen 3: Archivos de examenes individuales</strong></p>
  <img src="imagenes/03_examenes individuales.png" width="70%">
  <p><em>Fuente: Elaboración propia</em></p>
</div>

A continuación, muestro el contenido de uno de los examenes. Por favor note como los datos corresponden al alumno en cuestión. 

<div style="text-align: center;">
  <p><strong>Imagen 4: Ejemplo de examen individual</strong></p>
  <img src="imagenes/04_ejemplo_examen.png" width="70%">
  <p><em>Fuente: Elaboración propia</em></p>
</div>

Finalmente, obtenemos un archivo donde todos los examenes de un sólo profesor se unen en un sólo archivo. Como sólo tenemos tres profesores, sólo tenemos 3 archivos. Por favor nota que el nombre de los archivos corresponde con el número del profesor. Esto para identificar correctamente cada archivo. 

<div style="text-align: center;">
  <p><strong>Imagen 5: Archivos de examenes individuales</strong></p>
  <img src="imagenes/05_examenes_agrupados.png" width="70%">
  <p><em>Fuente: Elaboración propia</em></p>
</div>

El archivo de cada profesor quedaría de la siguiente manera. Por favor, nota que el número de páginas del archivo es de 266, pues este archvio agrupado contiene todos los examenes de ese profesor.

<div style="text-align: center;">
  <p><strong>Imagen 6: Archivos de examenes individuales</strong></p>
  <img src="imagenes/06_archivo_profesor.png" width="70%">
  <p><em>Fuente: Elaboración propia</em></p>
</div>

# Conclusiones. 

Como se mencionó al principio del documento, este código, u otros similares se pueden ocupar para hacer el llenado automático de diversos documentos a partir de una base de datos. Dependiendo de la plantilla y de la base que tengamos se pueden personalizar básicamente cualquier aspecto de un documento. Lo cual, podría mejorar la calidad de nuestro trabajo, y producir insumos personalizados de manera masiva, sólo limitados por nuestros propios recursos tecnológicos. 